In [5]:
from basenji import bed
import dataset_hic
from basenji import dataset
from basenji import plots
from basenji import seqnn
from basenji import trainer
from basenji import metrics
import tensorflow as tf
import h5py
import numpy as np
import seaborn as sns
import pandas as pd

In [6]:
np.__version__

'1.24.3'

In [2]:
### Load test data
target_file = "/results/basset_model/model_full/test_sequences_full_downsampled04p_noaug/targets.h5"
pred_file = "/results/basset_model/model_full/test_sequences_full_downsampled04p_noaug/preds.h5"
pred_avg = "/data/basset_predict/hic_sequences_full_downsampled04p/model_full/avg_predict.h5"
pred_hic = "/data/basset_predict/hic_sequences_full_downsampled04p/model_full/hic_only_predict.h5"

In [7]:
with_chr = "/data/basset_tfr/keep_chr1_downsampled5p"
wo_chr = "/data/basset_tfr/wo_chr1_downsampled5p"

In [4]:
pred_obj = h5py.File(pred_file)
pred_avg_obj = h5py.File(pred_avg)
pred_hic_obj = h5py.File(pred_hic)
tg_obj  = h5py.File(target_file)

In [5]:
pred_set = np.array(pred_obj.get('preds'))
pred_avg_set = np.array(pred_avg_obj.get('preds'))
pred_hic_set = np.array(pred_hic_obj.get('preds'))
tg_set = np.array(tg_obj.get('targets'))
pred_set[np.isnan(pred_set)] = 0
pred_avg_set[np.isnan(pred_avg_set)] = 0
pred_hic_set[np.isnan(pred_hic_set)] = 0

In [6]:
tg_set.shape

(4790, 1, 164)

In [7]:
tg_set_ti = tg_set[:, :,120]
tg_set_ti.shape
# sample every few bins (adjust to plot the # points I want)
ds_indexes = np.arange(0, pred_set.shape[1], 8)

# subset and flatten
tg_ti_flat = tg_set_ti[:, ds_indexes].flatten().astype('float32')
pred_ti_flat = pred_set[:, ds_indexes, 120].flatten().astype('float32')
pred_avg_ti_flat = pred_avg_set[:, ds_indexes, 120].flatten().astype('float32')
pred_hic_ti_flat = pred_hic_set[:, ds_indexes, 120].flatten().astype('float32')

In [8]:
tg_set_ti.shape

(4790, 1)

In [9]:
# call peaks
from scipy.stats import poisson
def ben_hoch(p_values):
  """ Convert the given p-values to q-values using Benjamini-Hochberg FDR. """
  m = len(p_values)

  # attach original indexes to p-values
  p_k = [(p_values[k], k) for k in range(m)]

  # sort by p-value
  p_k.sort()

  # compute q-value and attach original index to front
  k_q = [(p_k[i][1], p_k[i][0] * m // (i + 1)) for i in range(m)]

  # re-sort by original index
  k_q.sort()

  # drop original indexes
  q_values = [k_q[k][1] for k in range(m)]

  return q_values

test_targets_ti_lambda = np.mean(tg_ti_flat)
test_targets_pvals = 1 - poisson.cdf(
  np.round(tg_ti_flat) - 1, mu=test_targets_ti_lambda)
test_targets_qvals = np.array(ben_hoch(test_targets_pvals))
test_targets_peaks = test_targets_qvals < 0.01
test_targets_peaks_str = np.where(test_targets_peaks, 'Peak',
                                'Background')

In [ ]:
%matplotlib inline
import matplotlib
# matplotlib.use('PDF')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score

# cases_preds = [pred_ti_flat, pred_avg_ti_flat, pred_hic_ti_flat]
# aurocs = []
# aurprcs = []

# fig, axes = plt.subplots(ncols=3, figsizes=(15,5)) 

plt.figure()
fpr, tpr, _ = roc_curve(test_targets_peaks, pred_ti_flat)
auroc = roc_auc_score(test_targets_peaks, pred_ti_flat)
auprc = average_precision_score(test_targets_peaks, pred_ti_flat)
plt.title("Original + Hi-C data prediction") 
plt.plot(
  [0, 1], [0, 1], c='black', linewidth=1, linestyle='--', alpha=0.7)
plt.plot(fpr, tpr, c='black')
ax = plt.gca()
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.text(
  0.99, 0.02, 'AUROC %.3f' % auroc,
  horizontalalignment='right')  # , fontsize=14)
ax.grid(True, linestyle=':')
plt.show()

In [13]:
auprc

0.012480288280822517

In [10]:
dt_with_chr = dataset.SeqDataset(data_path,
                              split_label='test',
                              batch_size=64,
                              shuffle_buffer=8192,
                              mode='eval',
                              tfr_pattern=None)
dt_wo_chr

THE TFR FILES


In [ ]:
data_hic = dataset_hic.SeqDataset(data_path,
                              split_label='test',
                              batch_size=32,
                              shuffle_buffer=8192,
                              hic_length=25000,
                              cell_id=120,
                              mode='eval',
                              tfr_pattern=None)

In [20]:
import json
params_file = "/config/params_basset.json"
with open(params_file) as params_open:
    params = json.load(params_open)
params_model = params['model']
params_train = params['train']



# initialize model
seqnn_model = seqnn.SeqNN(params_model)
seqnn_model.restore("/data/basset_model/full_origin/model_best.h5", 0)
seqnn_model.build_ensemble(False, [])

test_preds = seqnn_model.predict(data).astype('float16')

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1344, 4)]    0           []                               
                                                                                                  
 stochastic_reverse_complement_  ((None, 1344, 4),   0           ['sequence[0][0]']               
 3 (StochasticReverseComplement   ())                                                             
 )                                                                                                
                                                                                                  
 stochastic_shift_3 (Stochastic  (None, 1344, 4)     0           ['stochastic_reverse_complement_3
 Shift)                                                          [0][0]']                   

2023-11-26 22:33:48.048859: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2023-11-26 22:33:48.788574: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [41]:
test_preds

array([[[2.6741e-03, 1.2703e-02, 2.8076e-03, ..., 3.4454e-02,
         1.1253e-02, 2.9602e-03]],

       [[1.1644e-03, 1.4687e-04, 9.2030e-04, ..., 2.0237e-03,
         9.8419e-04, 3.3855e-04]],

       [[3.8574e-02, 9.7084e-04, 3.7384e-02, ..., 8.2626e-03,
         6.3782e-02, 4.9858e-03]],

       ...,

       [[2.7294e-03, 3.2854e-04, 5.7640e-03, ..., 1.6022e-03,
         5.1041e-03, 8.5878e-04]],

       [[2.1545e-01, 4.3793e-03, 1.5527e-01, ..., 1.9165e-02,
         2.6459e-02, 2.4170e-02]],

       [[1.2054e-02, 1.0023e-03, 1.4359e-02, ..., 2.6722e-03,
         1.4477e-03, 1.5097e-03]]], dtype=float16)

In [11]:
test_targets = data.numpy(return_inputs=False)
test_targets.shape

(59884, 1, 164)

In [57]:
# flatten batch and sequence length
num_targets = test_preds.shape[-1]
y_true = tf.reshape(test_targets, (-1,num_targets))
y_pred = tf.reshape(test_preds, (-1,num_targets))


In [9]:
import json
params_file = "/config/params_basset.json"
with open(params_file) as params_open:
    params = json.load(params_open)
params_model = params['model']
params_train = params['train']

In [13]:
### Restore model
model_file = "/data/basset_model/full/model_best.h5"


seqnn_model = seqnn.SeqNN(params_model)
seqnn_model.restore(model_file, 0)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1344, 4)]    0           []                               
                                                                                                  
 stochastic_reverse_complement_  ((None, 1344, 4),   0           ['sequence[0][0]']               
 1 (StochasticReverseComplement   ())                                                             
 )                                                                                                
                                                                                                  
 stochastic_shift (StochasticSh  (None, 1344, 4)     0           ['stochastic_reverse_complement_1
 ift)                                                            [0][0]']                   

                                                                                                  
 reshape (Reshape)              (None, 1, 1792)      0           ['gelu_9[0][0]']                 
                                                                                                  
 dense (Dense)                  (None, 1, 768)       1376256     ['reshape[0][0]']                
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 1, 768)      3072        ['dense[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 dropout (Dropout)              (None, 1, 768)       0           ['batch_normalization_8[0][0]']  
                                                                                                  
 gelu_10 (

In [2]:
# load model
import layers

model_path_hic = "/data/basset_model/full_downsampled10p_hic_proper_384/model_best.h5"
model_file_hic = tf.keras.models.load_model(model_path_hic, custom_objects={'StochasticReverseComplement': layers.StochasticReverseComplement(),  
                                                                  'SwitchReverse': layers.SwitchReverse(), 'StochasticShift': layers.StochasticShift}, compile=False)

2023-12-03 21:08:25.209616: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-03 21:08:25.210151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-03 21:08:25.210634: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-03 21:08:25.211112: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-03 21:08:25.218055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [3]:
# load model trained on loop, not keras function
model_dir_hic = "/data/basset_model/full_downsampled10p_hic_no_dense_384/last_saved"
model_hic_from_dir = tf.keras.models.load_model(model_path_hic, custom_objects={'StochasticReverseComplement': layers.StochasticReverseComplement(),  
                                                                  'SwitchReverse': layers.SwitchReverse(), 'StochasticShift': layers.StochasticShift}, compile=False)

In [10]:
  def predict(model, seq_data, head_i=None, generator=False, **kwargs):
    """ Predict targets for SeqDataset. """
    # choose model

    dataset = getattr(seq_data, 'dataset', None)
    if dataset is None:
      dataset = seq_data

    if generator:
      return model.predict_generator(dataset, **kwargs)
    else:
      return model.predict(dataset, **kwargs)

In [5]:
data_path = "/data/basset_tfr/full_downsampled10p/"
data_hic = dataset_hic.SeqDataset(data_path,
                              split_label='test',
                              batch_size=32,
                              shuffle_buffer=8192,
                              hic_length=25000,
                              cell_id=120,
                              mode='eval',
                              tfr_pattern=None)

In [7]:
targets = data_hic.numpy(return_inputs=False, return_outputs=True)

<FlatMapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [ ]:
preds = predict(model_hic_from_dir, data_hic)

In [ ]:
model_from_path_hic.summary()

 reshape (Reshape)              (None, 1, 1792)      0           ['gelu_8[1][0]']                 
                                                                                                  
 dense (Dense)                  (None, 1, 768)       1376256     ['reshape[1][0]']                
                                                                                                  
 hic (InputLayer)               [(None, 25000, 4)]   0           []                               
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 1, 768)      3072        ['dense[1][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 1344, 4)     0           ['hic[0][0]']                    
 ingOpLamb

                                                                                                  
 tf.__operators__.getitem_25 (S  (None, 1344, 4)     0           ['hic[0][0]']                    
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__operators__.getitem_26 (S  (None, 1344, 4)     0           ['hic[0][0]']                    
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__operators__.getitem_27 (S  (None, 1344, 4)     0           ['hic[0][0]']                    
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__oper

 licingOpLambda)                                                                                  
                                                                                                  
 tf.__operators__.getitem_53 (S  (None, 1344, 4)     0           ['hic[0][0]']                    
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__operators__.getitem_54 (S  (None, 1344, 4)     0           ['hic[0][0]']                    
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__operators__.getitem_55 (S  (None, 1344, 4)     0           ['hic[0][0]']                    
 licingOpLambda)                                                                                  
          

                                                                 0]',                             
                                                                  'tf.__operators__.getitem_27[0][
                                                                 0]',                             
                                                                  'tf.__operators__.getitem_28[0][
                                                                 0]',                             
                                                                  'tf.__operators__.getitem_29[0][
                                                                 0]',                             
                                                                  'tf.__operators__.getitem_30[0][
                                                                 0]',                             
                                                                  'tf.__operators__.getitem_31[0][
          

                                                                  'model_1[9][0]',                
                                                                  'model_1[10][0]',               
                                                                  'model_1[11][0]',               
                                                                  'model_1[12][0]',               
                                                                  'model_1[13][0]',               
                                                                  'model_1[14][0]',               
                                                                  'model_1[15][0]',               
                                                                  'model_1[16][0]',               
                                                                  'model_1[17][0]',               
                                                                  'model_1[18][0]',               
          

                                                                  'dense_common[27][0]',          
                                                                  'dense_common[28][0]',          
                                                                  'dense_common[29][0]',          
                                                                  'dense_common[30][0]',          
                                                                  'dense_common[31][0]',          
                                                                  'dense_common[32][0]',          
                                                                  'dense_common[33][0]',          
                                                                  'dense_common[34][0]',          
                                                                  'dense_common[35][0]',          
                                                                  'dense_common[36][0]',          
          

In [10]:
# evaluate
loss_label = params_train.get('loss', 'poisson').lower()
spec_weight = params_train.get('spec_weight', 1)
loss_fn = trainer.parse_loss(loss_label, spec_weight=spec_weight)

In [11]:
def evaluate_model(model_object, seq_data, head_i=None, loss_label='poisson', loss_fn=None):
    """ Evaluate model on SeqDataset. """
    model = model_object

    # compile with dense metrics
    num_targets = model.output_shape[-1]

    if loss_fn is None:
        loss_fn = loss_label
    
    if loss_label == 'bce':
        model.compile(optimizer=tf.keras.optimizers.SGD(),
                    loss=loss_fn,
                    metrics=[metrics.SeqAUC(curve='ROC', summarize=False),
                             metrics.SeqAUC(curve='PR', summarize=False)])
    else:      
        model.compile(optimizer=tf.keras.optimizers.SGD(),
                    loss=loss_fn,
                    metrics=[metrics.PearsonR(num_targets, summarize=False),
                             metrics.R2(num_targets, summarize=False)])

    # evaluate
    return model.evaluate(seq_data.dataset)

In [16]:
# evaluate
import tensorflow as tf
test_loss, test_metric1, test_metric2 = evaluate_model(seqnn_model, data, loss_label=loss_label, loss_fn=loss_fn)

1872/1872 [==============================] - 27s 14ms/step - loss: 0.0761 - auroc: 0.9318 - auprc: 0.5003


(164,)

In [16]:
import tensorflow as tf
test_loss_1, test_metric11, test_metric22 = evaluate_model(model_from_path, data, loss_label=loss_label, loss_fn=loss_fn)

75/75 [==============================] - 2s 14ms/step - loss: 0.1286 - auroc: 0.7531 - auprc: 0.1589


In [11]:
if loss_label == 'bce':
    print('Test AUROC:        %7.5f' % test_metric1.mean())
    print('Test AUPRC:        %7.5f' % test_metric2.mean())

Test AUROC:        0.48162
Test AUPRC:        0.03432


In [12]:
AUROC: 0.76010
AUPRC: 0.18207

In [7]:
data_path_5p_wo_chr = "/home/dkokot/Basenji_HiC/data/basset_tfr/wo_chr1_downsampled5p"
data_hic_5p_wo_chr = dataset_hic.SeqDataset(data_path_5p_wo_chr,
                              split_label='test',
                              batch_size=32,
                              shuffle_buffer=8192,
                              hic_length=25000,
                              cell_id=120,
                              mode='eval',
                              tfr_pattern=None)

In [8]:
data_path_5p_keep_chr = "/home/dkokot/Basenji_HiC/data/basset_tfr/keep_chr1_downsampled5p"
data_hic_5p_keep_chr = dataset_hic.SeqDataset(data_path_5p_keep_chr,
                              split_label='test',
                              batch_size=32,
                              shuffle_buffer=8192,
                              hic_length=25000,
                              cell_id=120,
                              mode='eval',
                              tfr_pattern=None)

In [ ]:
data_path_5p = "/home/dkokot/Basenji_HiC/data/basset_tfr/full_downsampled5p"
data_hic_5p = dataset_hic.SeqDataset(data_path_5p,
                              split_label='test',
                              batch_size=32,
                              shuffle_buffer=8192,
                              hic_length=25000,
                              cell_id=120,
                              mode='eval',
                              tfr_pattern=None)

In [9]:
import layers

model_dir_hic_5p_wo1 = "/home/dkokot/Basenji_HiC/data/basset_model/wo_chr1_5p/last_saved"
model_hic_from_dir_5p_wo1 = tf.keras.models.load_model(model_dir_hic_5p_wo1, custom_objects={'StochasticReverseComplement': layers.StochasticReverseComplement(),  
                                                                  'SwitchReverse': layers.SwitchReverse(), 'StochasticShift': layers.StochasticShift, 
                                                                  'GELU': layers.GELU(), 'GELU_HIC': layers.GELU_HIC, 'GELU_FINAL': layers.GELU_FINAL}, compile=False)

In [17]:
# evaluate
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
test_loss_hic, test_metric1_hic, test_metric2_hic = evaluate_model(model_hic_from_dir, data_hic, loss_label=loss_label, loss_fn=loss_fn)

3743/3743 [==============================] - 1572s 414ms/step - loss: 0.0982 - auroc: 0.8850 - auprc: 0.4291


In [11]:
test_metric1_hic

array([0.884969], dtype=float32)

In [11]:
# 5p wo chr1 test
test_loss_hic, test_metric1_hic, test_metric2_hic = evaluate_model(model_hic_from_dir_5p_wo1, data_hic_5p_wo_chr, loss_label=loss_label, loss_fn=loss_fn)

2022-09-14 10:39:55.203320: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2022-09-14 10:39:55.569741: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1712/1712 [==============================] - 734s 415ms/step - loss: 0.0935 - auroc: 0.9040 - auprc: 0.4965


In [12]:
# 5p only chr1 test
test_loss_hic, test_metric1_hic, test_metric2_hic = evaluate_model(model_hic_from_dir_5p_wo1, data_hic_5p_keep_chr, loss_label=loss_label, loss_fn=loss_fn)

160/160 [==============================] - 87s 412ms/step - loss: 0.1275 - auroc: 0.8882 - auprc: 0.5009


# Testing how trained model on data without one chromosome predicts sequences from that chromosome

In [ ]:
# load model
import layers

model_dir_hic_5p_wo1 = "/home/dkokot/Basenji_HiC/data/basset_model/wo_chr1_hic/model_best.h5"
model_hic_from_dir_5p_wo1 = tf.keras.models.load_model(model_dir_hic_5p_wo1, custom_objects={'StochasticReverseComplement': layers.StochasticReverseComplement(),  
                                                                  'SwitchReverse': layers.SwitchReverse(), 'StochasticShift': layers.StochasticShift, 
                                                                  'GELU': layers.GELU(), 'GELU_HIC': layers.GELU_HIC, 'GELU_FINAL': layers.GELU_FINAL}, compile=False)

# Evaluate model on test data from without chr set

In [12]:
# 5p wo chr1 test hic
test_loss_hic, test_metric1_hic, test_metric2_hic = evaluate_model(model_hic_from_dir_5p_wo1, data_hic_5p_wo_chr, loss_label=loss_label, loss_fn=loss_fn)

2023-04-04 16:30:58.905513: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-04-04 16:31:00.149843: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1712/1712 [==============================] - 861s 473ms/step - loss: 0.0883 - auroc: 0.9092 - auprc: 0.4990


# Evalute model on test data from set with only chromosome that is not inluded in training

In [14]:
# 5p keep chr1 test hic
test_loss_hic_keep_chr1, test_metric1_hic_keep_chr1, test_metric2_hic_keep_chr1 = evaluate_model(model_hic_from_dir_5p_wo1, data_hic_5p_keep_chr, loss_label=loss_label, loss_fn=loss_fn)

160/160 [==============================] - 126s 475ms/step - loss: 0.1280 - auroc: 0.8879 - auprc: 0.5070
